# US Accidents - Severity Prediction - SparkML

## Team Members
- Aditya Kamble
- Sidharth Panda

## Import Dependencies

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

Intel(R) Data Analytics Acceleration Library (Intel(R) DAAL) solvers for sklearn enabled: https://intelpython.github.io/daal4py/sklearn.html


In [28]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

## Data Preparation

In [3]:
sparkContext = SparkContext.getOrCreate()
spark = SparkSession(sparkContext)

In [4]:
data_df = spark.read.csv('data/US_Accidents_Dec19.csv', header = True, inferSchema = True)
data_df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- TMC: double (nullable = true)
 |-- Severity: integer (nullable = true)
 |-- Start_Time: timestamp (nullable = true)
 |-- End_Time: timestamp (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- Start_Lng: double (nullable = true)
 |-- End_Lat: double (nullable = true)
 |-- End_Lng: double (nullable = true)
 |-- Distance(mi): double (nullable = true)
 |-- Description: string (nullable = true)
 |-- Number: double (nullable = true)
 |-- Street: string (nullable = true)
 |-- Side: string (nullable = true)
 |-- City: string (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Timezone: string (nullable = true)
 |-- Airport_Code: string (nullable = true)
 |-- Weather_Timestamp: timestamp (nullable = true)
 |-- Temperature(F): double (nullable = true)
 |-- Wind_Chill(F): double (n

In [5]:
features_imp = ['TMC',
                'Start_Lng',
                'Start_Lat',
                'Distance(mi)',
                'Temperature(F)',
                'Humidity(%)',
                'Pressure(in)',
                'State',
                'Severity']

In [6]:
numerical_feartures = ['TMC',
                'Start_Lng',
                'Start_Lat',
                'Distance(mi)',
                'Temperature(F)',
                'Humidity(%)',
                'Pressure(in)']

In [7]:
data_df = data_df.select(features_imp)
data_df_cols = data_df.columns
data_df.printSchema()

root
 |-- TMC: double (nullable = true)
 |-- Start_Lng: double (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- Distance(mi): double (nullable = true)
 |-- Temperature(F): double (nullable = true)
 |-- Humidity(%): double (nullable = true)
 |-- Pressure(in): double (nullable = true)
 |-- State: string (nullable = true)
 |-- Severity: integer (nullable = true)



In [8]:
categoricalColumns = ['State']
stages = []

In [9]:
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

In [10]:
label_stringIdx = StringIndexer(inputCol = 'Severity', outputCol = 'label')
stages += [label_stringIdx]

In [11]:
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numerical_feartures
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [12]:
stages

[StringIndexer_c5e3cb5dacb3,
 OneHotEncoderEstimator_764827334c06,
 StringIndexer_51e43345cac4,
 VectorAssembler_a39eac2d503a]

In [13]:
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(data_df)

In [14]:
data_df = pipelineModel.transform(data_df)
selectedCols = ['label', 'features'] + data_df_cols

In [15]:
data_df = data_df.select(selectedCols)
data_df.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- TMC: double (nullable = true)
 |-- Start_Lng: double (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- Distance(mi): double (nullable = true)
 |-- Temperature(F): double (nullable = true)
 |-- Humidity(%): double (nullable = true)
 |-- Pressure(in): double (nullable = true)
 |-- State: string (nullable = true)
 |-- Severity: integer (nullable = true)



In [17]:
train, test = data_df.randomSplit([0.7, 0.3], seed = 2018)

In [25]:
train.count, test.count

(<bound method DataFrame.count of DataFrame[label: double, features: vector, TMC: double, Start_Lng: double, Start_Lat: double, Distance(mi): double, Temperature(F): double, Humidity(%): double, Pressure(in): double, State: string, Severity: int]>,
 <bound method DataFrame.count of DataFrame[label: double, features: vector, TMC: double, Start_Lng: double, Start_Lat: double, Distance(mi): double, Temperature(F): double, Humidity(%): double, Pressure(in): double, State: string, Severity: int]>)

In [29]:
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')

In [30]:
rfModel = rf.fit(train)

Py4JJavaError: An error occurred while calling o376.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 12.0 failed 1 times, most recent failure: Lost task 0.0 in stage 12.0 (TID 97, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$4: (struct<StateclassVec:struct<type:tinyint,size:int,indices:array<int>,values:array<double>>,TMC:double,Start_Lng:double,Start_Lat:double,Distance(mi):double,Temperature(F):double,Humidity(%):double,Pressure(in):double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.sort_addToSorter_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.

In [ ]:
predictions = rfModel.transform(test)
predictions.select('age', 'job', 'label', 'rawPrediction', 'prediction', 'probability').show(10)